In [1]:
!pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 22.6 MB/s eta 0:00:00


In [1]:
import torch

# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

Using device: cuda



In [2]:
import pandas as pd

# Load the dataset into a pandas DataFrame
df = pd.read_csv('yt_comments.csv')
df.head()

,Unnamed: 0,Video ID,Comment,Likes,Sentiment
0,0,wAZZ-UWGVHI,Let's not forget that Apple Pay in 2014 requir...,95.0,1.0
1,1,wAZZ-UWGVHI,Here in NZ 50% of retailers don’t even have co...,19.0,0.0
2,2,wAZZ-UWGVHI,I will forever acknowledge this channel with t...,161.0,2.0
3,3,wAZZ-UWGVHI,Whenever I go to a place that doesn’t take App...,8.0,0.0
4,4,wAZZ-UWGVHI,"Apple Pay is so convenient, secure, and easy t...",34.0,2.0


In [3]:
print(df.columns)
print('\n\n')
print(df['Sentiment'].value_counts())
print('\n\n')
print(df.info())

Index(['Unnamed: 0', 'Video ID', 'Comment', 'Likes', 'Sentiment'], dtype='object')



2.0    11432
1.0     4639
0.0     2338
Name: Sentiment, dtype: int64



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18409 entries, 0 to 18408
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  18409 non-null  int64  
 1   Video ID    18409 non-null  object 
 2   Comment     18408 non-null  object 
 3   Likes       18409 non-null  float64
 4   Sentiment   18409 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 719.2+ KB
None


In [4]:
df['Sentiment'] = df['Sentiment'].astype(int)
df = df.drop(['Unnamed: 0', 'Video ID', 'Likes'], axis=1)
df.rename(columns = {'Comment':'comment', 
                              'Sentiment':'sentiment'}, inplace = True)
df = df.dropna()

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=3)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.out_proj.bias

In [6]:
from sklearn.model_selection import train_test_split

X = df["comment"].to_list()
y = df["sentiment"].to_list()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Show the number of samples in the training and testing sets
print('Number of samples in the training set:', len(X_train))
print('Number of samples in the validation set:', len(X_val))

Number of samples in the training set: 14726
Number of samples in the validation set: 3682


In [7]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [9]:
X_train_tokenized.keys()     

dict_keys(['input_ids', 'attention_mask'])

In [10]:
print(X_train_tokenized['input_ids'][0])
print(X_train_tokenized['attention_mask'][0])
print(len(X_train_tokenized['input_ids'][0]))
print(len(X_train_tokenized['attention_mask'][0]))

[0, 100, 6573, 1690, 636, 20093, 15, 155, 1178, 246, 6, 142, 38, 657, 5, 356, 4, 5053, 2671, 35788, 6, 5, 1280, 9, 24977, 95, 817, 24, 45, 966, 24, 7, 162, 6, 7, 171, 14, 64, 120, 5009, 6, 8, 8119, 24977, 1516, 30413, 55, 19887, 4, 38, 74, 1153, 619, 8225, 114, 5, 24977, 31, 15707, 44406, 39948, 3559, 75, 25, 205, 25, 51, 32, 6, 98, 38, 74, 202, 185, 10, 1690, 636, 20093, 155, 1178, 246, 81, 10, 26334, 1672, 6, 576, 5, 1973, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [11]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [13]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import numpy as np

def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [14]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=20,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=200,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
200,0.745500
400,0.403800
600,0.414000
800,0.356100
1000,0.290100
1200,0.269200
1400,0.261000
1600,0.181200
1800,0.158500
2000,0.151800


TrainOutput(global_step=3685, training_loss=0.20983880294064816, metrics={'train_runtime': 3224.1916, 'train_samples_per_second': 22.837, 'train_steps_per_second': 1.143, 'total_flos': 9753748504197120.0, 'train_loss': 0.20983880294064816, 'epoch': 5.0})

In [17]:
trainer.save_model('ft_distilroberta')

In [ ]:
trainer.evaluate()

In [18]:
!mkdir tr_model
!mkdir tr_tokenizer

In [19]:
model.save_pretrained("./tr_model")
tokenizer.save_pretrained('./tr_tokenizer')

('./tr_tokenizer/tokenizer_config.json',
 './tr_tokenizer/special_tokens_map.json',
 './tr_tokenizer/vocab.json',
 './tr_tokenizer/merges.txt',
 './tr_tokenizer/added_tokens.json',
 './tr_tokenizer/tokenizer.json')

In [21]:
!pip install transformers[onnx]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.3/442.3 KB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 KB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 90.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 12.1 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 23.3.3
    Uninstalling flatbuffers-23.3.3:
      Successfully uninstalled flatbuffers-23.3.3
  At

In [28]:
import transformers
from pathlib import Path
from transformers.onnx import FeaturesManager

feature = "sequence-classification"
# load config
model_kind, model_onnx_config = FeaturesManager.check_supported_model_or_raise(model, feature=feature)
onnx_config = model_onnx_config(model.config)

# export
onnx_inputs, onnx_outputs = transformers.onnx.export(
        preprocessor=tokenizer,
        model=model,
        config=onnx_config,
        opset=13,
        output=Path("ft-distilroberta.onnx")
)

In [29]:
onnx_inputs

['input_ids', 'attention_mask']

In [30]:
onnx_outputs

['logits']

In [31]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install onnxruntime-gpu

In [16]:
# first create session 
import onnxruntime 
onnx_session = onnxruntime.InferenceSession('ft-distilroberta.onnx', 
                                            providers=['CPUExecutionProvider']
                                            )

In [17]:
samples_list = X_val[:32]

In [21]:
import numpy as np

inputs = tokenizer(samples_list, max_length=512, padding=True, truncation=True, return_tensors='pt')
feed = dict(input_ids=np.array(inputs["input_ids"]).astype("int64"),
            attention_mask=np.array(inputs["attention_mask"]).astype("int64"),)

In [31]:
outputs = onnx_session.run(None, feed)
predictions = torch.nn.functional.softmax(torch.from_numpy(outputs[0]), dim=-1)
predictions = predictions.cpu().detach().numpy()
labels = ['NEG', 'NUE', 'POS']
res = [labels[x] for x in np.argmax(predictions, axis=1)]

In [ ]:
labels = ['NEG', 'NUE', 'POS']
target = [labels[x] for x in y_val[:32]]

for i, j in zip(res, target):
  print(i, j)

In [ ]:
# Comparison

In [8]:
import numpy as np

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
# model = AutoModelForSequenceClassification.from_pretrained("distilroberta-base", num_labels=3)
model = AutoModelForSequenceClassification.from_pretrained("./tr_model")

In [10]:
# first create session 
import onnxruntime 
onnx_session = onnxruntime.InferenceSession('ft-distilroberta.onnx', 
                                            providers=['CPUExecutionProvider']
                                            )

In [11]:
samples_list = X_val[:320]
# inputs = tokenizer(samples_list, max_length=512, padding=True, truncation=True, return_tensors='pt')
# feed = dict(input_ids=np.array(inputs["input_ids"]),
#             attention_mask=np.array(inputs["attention_mask"]),)

In [12]:
import time

onnxtime = []
pytorchtime = []

for i in range(0, 320, 16):
  inputs = tokenizer(samples_list[i:i+16], max_length=512, padding=True, truncation=True, return_tensors='pt')
  
  start = time.time()
  feed = dict(input_ids=np.array(inputs["input_ids"]),
            attention_mask=np.array(inputs["attention_mask"]),)
  onnx_outputs = onnx_session.run(None, feed)
  end = time.time()

  onnxtime.append((end-start) * 10**3)

  start = time.time()
  pytorch_outputs = model(**inputs)
  end = time.time()

  pytorchtime.append((end-start) * 10**3)

print("onnx TOTAL execution time:", sum(onnxtime), "ms")
print("pytorch TOTAL execution time:", sum(pytorchtime), "ms")

onnx TOTAL execution time: 85787.23049163818 ms
pytorch TOTAL execution time: 96017.657995224 ms


In [13]:
!cp -r ./ft_distilroberta ./drive/MyDrive/yt_comments_sentiment
!cp -r ft-distilroberta.onnx ./drive/MyDrive/yt_comments_sentiment